# Modules, functions

In [ ]:
import numpy as np
import pandas as pd
import multiprocessing

import matplotlib.pyplot as plt
import bokeh
import bokeh.io
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

# init_notebook_mode()

import seaborn as sns

import re
import math
import copy

from collections import defaultdict
import csv
import itertools
import datetime 
from datetime import datetime
import time
import dateutil.parser
import pickle
import random

import gc
import zipfile
import sys, getopt
import os

from IPython.core.interactiveshell import InteractiveShell
from io import StringIO

import dask.dataframe as dd
#from chest import Chest

InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last"

# Magic function to make matplotlib inline
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

import dask.dataframe as ddf
import dask.array as da

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 800)

import scipy

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.tsatools import detrend

import datetime as dt

# Data

In [ ]:
dat0 = pd.read_csv('data/ch4k_df.csv')
ref_dat0 = pd.read_csv('data/Article reference data.csv', low_memory = False, error_bad_lines = False, 
                       usecols = ['article_no', 'model_no', 'art_desc', 'sports_cat_desc', 'rmh_cat_desc', 
                                  'franchise', 'gender_desc', 'age_group_desc', 'prod_grp_desc', 'prod_type_desc',
                                  'brand_desc', 'bus_unit_desc', 'rmh_cat_desc'])

In [ ]:
# Remove clearance transactions!!

dat0['clearance'] = dat0.clearance.fillna(0) 

dat0['net_qty'] = (1 - dat0.clearance)*dat0.net_qty

In [ ]:
dat0.head()

# Aggregate to Season

In [ ]:
dat = dat0.copy()

dat = dat[(dat.net_qty != 0) & (dat.season.isin(['FW15', 'FW16', 'FW17', 'FW18', 'FW19']))].round()

dat = dat[['article_number', 'season', 'season_net_qty', 'art_desc', 'sports_cat_desc',
       'rmh_cat_desc', 'franchise', 'gender_desc', 'age_group_desc',
       'prod_grp_desc', 'prod_type_desc', 'price', 'margin', 'cost']].drop_duplicates()

dat = (
    dat.
    sort_values(['article_number', 'season']).
    drop_duplicates(subset = 'article_number').
    set_index('article_number').
    dropna()
      )

dat[['price', 'margin', 'cost']] = dat[['price', 'margin', 'cost']].astype('int')

In [ ]:
len(dat.index.unique())
dat.shape
pd.crosstab(index = dat.season, columns = 'count').head() # new articles by season
# dat[dat.article_number == '015110']
dat.head()
dat.loc['015110']

In [ ]:
# Algorithm
# 1. select artice
# 2. select category
# 3. filter by same level of category
# 4. filter by similar price
# 5. retrieve net quantites
# 6. repeat 2 - 5 for all categories

In [ ]:
# 1. 015110	
# 2. 'sports_cat_desc'
# 3. 'sports_cat_desc' == 'FOOTBALL/SOCCER'
# 4. 70 < price < 110
# 5. net_qtys = net_qtys.append


In [ ]:
int(9 - .2*9)
int(p) in range(int(p - 0.2*p), int(p + 0.2*p))

In [ ]:
dat.loc['015110']


In [ ]:
net_qtys = pd.Series()

for a in ['015110']:
    for c in ['sports_cat_desc', 'rmh_cat_desc', 'franchise', 'gender_desc', 'age_group_desc', 'prod_grp_desc', 'prod_type_desc']: 
        a_c_l = dat.loc[a, c] # article_category_level
        print(c, ' = ', a_c_l)
        
        p = dat.loc[a, 'price']
        print('price = ', p)
        
        dat_a = dat[(dat[c] == a_c_l) & ([price in range(int(p - 0.2*p), int(p + 0.2*p)) for price in dat.price])]
        print(dat_a.shape)
        print()
        
        net_qtys = net_qtys.append(dat_a.season_net_qty)


In [ ]:
net_qtys.shape
len(net_qtys)
net_qtys.describe()
        

In [ ]:
plt.hist(net_qtys, bins = [0, 100, 250, 500, 750, 1000, 1250, 1500, 1750, 2000, 20000], density = True)
# (array([4.52881587e-03, 1.62850089e-03, 5.75466076e-04, 2.46360773e-04,
#         1.34161914e-04, 6.72512131e-05, 4.98850770e-05, 3.67753469e-05,
#         1.90686984e-05, 1.14450025e-06]),
#  array([    0,   100,   250,   500,   750,  1000,  1250,  1500,  1750,
#          2000, 20000]),



In [ ]:
import numpy as np
from statsmodels.distributions.empirical_distribution import ECDF

ecdf = ECDF(net_qtys)

ecdf([100, 500, 1000, 2000])


# EDA Plots

In [ ]:
# ---- Plot -----
# a = np.random.choice(preds.article_number.unique(), size = 1, replace = False)[0]
aoi = 'G26535'
a = aoi

dat_a = preds[preds.article_number == a][['week', 'net_qty', 'corrected', 'y_hat']]
        

plt.rcParams["figure.figsize"] = [10,7]
dat_a.sort_values('week').set_index('week').plot(linewidth = 3)
dat_a.sort_values('week').set_index('week').round()

dat_a[['net_qty', 'corrected', 'y_hat']].apply(np.sum).round()

preds_season.reset_index()[preds_season.index == a]

dat_aoi = dat0[dat0.article_number == aoi].copy()

dat_aoi = pd.merge(
    pd.DataFrame(dat_aoi.groupby(['year', 'week'])['net_qty'].sum()).reset_index(),
    dat_aoi[['year', 'week']].drop_duplicates()
)

dat_aoi.year = [str(x) for x in dat_aoi.year]
dat_aoi.week = [str(x) for x in dat_aoi.week]
dat_aoi['date'] = [dt.datetime.strptime(x[0] + '-' + x[1] + '-1', "%Y-%W-%w") for x in zip(dat_aoi.year, dat_aoi.week)]

plt.rcParams["figure.figsize"] = [10,7]
dat_aoi[['date', 'net_qty']].set_index('date').plot(linewidth = 4)

# dat_aoi

# Overbuy

In [ ]:
from functools import partial
from scipy import optimize
from scipy import integrate
import scipy.stats as stats

In [ ]:
# Loss --- demand, buy, margin, cost
def L(d, b, margin, cost):
    if d > b:    # CANNOT satisfy demand
        return (d - b)*margin
    
    elif d <= b: # CAN satisfy demand
        return (b - d)*cost
    
    else:
        print('Error')

# E[L | buy, article_mean, article_sd, article_margin, article_cost]
def EL(net_qtys, margin, cost, b):
    return sum([L(x, b, margin, cost) for x in net_qtys])/len(net_qtys)

def minimize_EL(net_qtys, margin, cost):
    p = partial(EL, net_qtys, margin, cost) # Make EL function of only one var
    mu = np.mean(net_qtys)
    buy_opt = optimize.minimize_scalar(p, bounds = (mu, mu + 2*np.std(net_qtys)))
    return int(buy_opt['x']) # optimal buy quantity

In [ ]:
# Profit
def P(d, margin, cost, b):
    if d > b:    # CANNOT satisfy demand
        return b*margin
    
    elif d <= b: # CAN satisfy demand
        return d*margin - (b - d)*cost
    
    else:
        print('Error')

def EP(net_qtys, margin, cost, b):
    return -sum([P(x, margin, cost, b) for x in net_qtys])/len(net_qtys)

def maximize_EP(net_qtys, margin, cost):
    p = partial(EP, net_qtys, margin, cost) # Make EL function of only one var
    mu = np.mean(net_qtys)
    buy_opt = optimize.minimize_scalar(p, bounds = (mu, mu + 2*np.std(net_qtys)))
    return int(buy_opt['x']) # optimal buy quantity

In [ ]:
# one off e.g.s

# Minimize Loss
l = partial(EL, net_qtys, 63, 30) # Make EL function of only one var
[print(x, ':', round(l(x))) for x in range(0, 600, 50)]

# Maximize Profit
p = partial(EP, net_qtys, 63, 30)
[print(x, ':', round(p(x))) for x in range(0, 600, 50)]


In [ ]:
minimize_EL(net_qtys, 63, 30)
                          
maximize_EP(net_qtys, 63, 30)

# HUZZAH!!!!!


In [ ]:


# --- Overbuy statistics ---
# preds_season['pct_overbuy'] = (preds_season.Opt_Ovb - preds_season.y_hat)/preds_season.y_hat*100
# b = np.array([0, 1000, 2000, 5000, 10000, 50000])
# preds_season['bins'] = pd.cut(preds_season.y_hat, bins = b)
# preds_season.groupby('bins')['pct_overbuy'].describe().round()
